In [1]:
import keras
print('Keras version:', keras.__version__)

import sys
import os
import tensorflow as tf
print('TensorFlow version:', tf.__version__) 
from keras import backend as K

Using TensorFlow backend.


Keras version: 2.2.2
TensorFlow version: 1.8.0


In [2]:
sess = tf.Session()
K.set_session(sess)
from keras.models import model_from_json

DEVICE = "/cpu:0"  # /use cpu:0 for cpu only or /gpu:0


In [2]:
working_dir = os.getcwd()
print("Working directory is:",working_dir)
sys.path.append(working_dir)  # To find local version of the library
print("    ")

Working directory is: C:\Users\is95217\source\turkishplate


In [7]:
win=True
if win:
    ROOT_DIR = os.path.join("C:/Users/is95217/source/turkishplate") #win
    DATA_DIR = "C:/Users/is95217/data/Original_15Kcrop_distributed" #win
    MODEL_DIR = 'C:/Users/is95217/source/turkishplate/model' 
else:    
    ROOT_DIR = os.path.join("/home/user/Source/dev/Evrim/Turkishplate/") #linux
    DATA_DIR = "/home/user/Source/data/dataEvrim//plaka_out_distr"  #augmented plate directory
    MODEL_DIR = "/home/user/Source/dev/Evrim/Turkishplate//model"
sys.path.append(ROOT_DIR + "/Python")

print("Data dir:", DATA_DIR)
print("Source code dir:", ROOT_DIR)
print("Model dir:",MODEL_DIR)

Data dir: C:/Users/is95217/data/Original_15Kcrop_distributed
Source code dir: C:/Users/is95217/source/turkishplate
Model dir: C:/Users/is95217/source/turkishplate/model


In [8]:
sys.path.append(os.path.dirname(os.path.realpath('Modelop')) + "/Python")
import Modelop as p

In [9]:
import concurrent.futures
import numpy as np
import cv2
import keras.backend as K
from keras.layers import Dense
from keras.models import Sequential

import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.backend.tensorflow_backend import set_session

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
tf_config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
    

# LOAD MODEL 

In [10]:
graph_detection = tf.Graph()

with graph_detection.as_default():
    session_detection = tf.Session(graph = graph_detection, config=tf_config) 
    with session_detection.as_default():
        #load model
        loaded_model=p.load_model(MODEL_DIR +'/model_aug3') #mix pics from port and augmentation

# one plate predict

In [13]:
#img_filepath = 'C:/Users/is95217/data/plaka/' + '78BT701.png'
#img_filepath = 'C:/Users/is95217/data/plaka2/' + 'Test2.jpg'
img_filepath = DATA_DIR + '/test/81VY339.jpg' #ok
img_filepath = DATA_DIR + '/test/78O2555.jpg' # ok
#img_filepath = '/home/user/Source/data/dataEvrim/Original_15Kcrop_distr/test/' + '1#81FC650#12042018#085508.jpg' #ok
#img_filepath = '/home/user/Source/data/dataEvrim/Original_15Kcrop_distr/test/' + '1#78BN724#31032018#092749.jpg' #not ok 78bn777

#img_filepath = DATA_DIR + '/test/81U3872.jpg' 

image = cv2.imread(img_filepath)

In [14]:
img_filepath

'C:/Users/is95217/data/Original_15Kcrop_distributed/test/78O2555.jpg'

In [15]:

with graph_detection.as_default():
     with session_detection.as_default():
            predicted_text = p.predict_single_plate(session_detection,loaded_model,image)#input directory of pic
            print(predicted_text)

error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


#  Full test set predict

In [16]:
cnt_test= len(os.listdir(DATA_DIR + '/test'))
img_filepath = DATA_DIR + '/test/' 

In [17]:
cnt_test

1540

In [18]:
matched=0
start='#'
end='#'
for fname in os.listdir(DATA_DIR + '/test'):
     
    image = cv2.imread(img_filepath + fname)
    if p.check_word(fname):                
        fn= (fname.split(start))[1].split(end)[0] 
    else:
        fn = fname.split('.')   
        fn= fn[0]
        
    with graph_detection.as_default():
         with session_detection.as_default():
            predicted_text = p.predict_single_plate(session_detection,loaded_model,image)#input directory of pic
            
    
    
    if ''.join(predicted_text) == fn:
        matched +=1
    """
    else:
        print('notmatched',''.join(predicted_text),'-',fn)
        #print('matched')
        
    """

In [19]:
print("matched=",matched,",total=", cnt_test ,",accuracy:", matched/cnt_test*100)
 

matched= 1395 ,total= 1540 ,accuracy: 90.5844155844156
